# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchensemble import VotingClassifier
from torch.utils.data import Subset, DataLoader
import numpy as np
# Extras
import time
import os
# Torch Ensemble imports
from torchensemble import VotingClassifier  # voting is a classic ensemble strategy
from torchensemble.utils import io

In [2]:
# params
# Set training parameters
batch_size = 64
learning_rate = 0.001
weight_decay = 1e-4
epochs = 30

base_estimator = nn.Sequential(
    nn.Flatten(),            # Flatten the 28x28 input images to a vector of 784
    nn.Linear(28 * 28, 256), # First fully connected layer
    nn.ReLU(),               # Activation function
    nn.Linear(256, 128),     # Second fully connected layer
    nn.ReLU(),               # Activation function
    nn.Linear(128, 10)       # Output layer for 10 classes
)
# If using GPU, transfer data to the GPU inside fit() and evaluate() loops
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Set the base estimator to the device
base_estimator.to(device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=128, bias=True)
  (4): ReLU()
  (5): Linear(in_features=128, out_features=10, bias=True)
)

In [3]:
# Load data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize using MNIST mean and std
])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Number of samples per class
samples_per_class = 2000

# Get all targets
all_targets = train_dataset.targets.numpy()

# Initialize list to store indices
subset_indices = []

# For each class, get the first 'samples_per_class' indices
for class_label in range(10):  # MNIST has 10 classes
    class_indices = np.where(all_targets == class_label)[0][:samples_per_class]
    subset_indices.extend(class_indices)

# Create the subset
balanced_subset = Subset(train_dataset, subset_indices)

# Create a DataLoader for the balanced subset
balanced_training_loader = DataLoader(balanced_subset, batch_size=batch_size, shuffle=True)

In [4]:
total= 0
for batch in balanced_training_loader:
    total+=len(batch[0])
total

20000

In [5]:
# Define the ensemble
ensemble = VotingClassifier(
    estimator=base_estimator,  # here is your deep learning model
    n_estimators=5,           # number of base estimators
    cuda=torch.cuda.is_available(),  # Enable GPU if available
)

# Set the criterion
criterion = nn.CrossEntropyLoss()  # training objective
ensemble.set_criterion(criterion)

# Set the optimizer
ensemble.set_optimizer(
    "Adam",                       # type of parameter optimizer
    lr=learning_rate,             # learning rate of parameter optimizer
    weight_decay=weight_decay,    # weight decay of parameter optimizer
)

# Set the learning rate scheduler
ensemble.set_scheduler(
    "CosineAnnealingLR",          # type of learning rate scheduler
    T_max=epochs,                 # number of epochs for decay
)

# Train the ensemble

In [6]:
start_time = time.time()
ensemble.fit(
    balanced_training_loader,
    epochs=epochs
)
end_time = time.time()
print(f"Time taken: {end_time-start_time:.3f} seconds")

Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 2.30704 | Correct: 6/64
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 0.31013 | Correct: 58/64
Estimator: 000 | Epoch: 000 | Batch: 200 | Loss: 0.27953 | Correct: 60/64
Estimator: 000 | Epoch: 000 | Batch: 300 | Loss: 0.19611 | Correct: 61/64
Estimator: 001 | Epoch: 000 | Batch: 000 | Loss: 2.29103 | Correct: 9/64
Estimator: 001 | Epoch: 000 | Batch: 100 | Loss: 0.27512 | Correct: 60/64
Estimator: 001 | Epoch: 000 | Batch: 200 | Loss: 0.39145 | Correct: 57/64
Estimator: 001 | Epoch: 000 | Batch: 300 | Loss: 0.18276 | Correct: 61/64
Estimator: 002 | Epoch: 000 | Batch: 000 | Loss: 2.29860 | Correct: 6/64
Estimator: 002 | Epoch: 000 | Batch: 100 | Loss: 0.28493 | Correct: 58/64
Estimator: 002 | Epoch: 000 | Batch: 200 | Loss: 0.17019 | Correct: 62/64
Estimator: 002 | Epoch: 000 | Batch: 300 | Loss: 0.31302 | Correct: 59/64
Estimator: 003 | Epoch: 000 | Batch: 000 | Loss: 2.30110 | Correct: 6/64
Estimator: 003 | Epoch: 000 | Batch: 100 |

# Saving and Loading Trained Ensemble Model

In [7]:
# Assuming ensemble_model is already fitted
save_dir = 'Trained_models/'
model_filename = "VotingClassifier_Sequential_3_ckpt.pth"
save_path = os.path.join(save_dir, model_filename)

# Save model state dict
torch.save(ensemble.state_dict(), save_path)

In [8]:
# Instantiate the model with the same configuration as the original
ensemble_model = VotingClassifier(
    estimator=base_estimator,
    n_estimators=3,
    cuda=True
)

# Load the saved state dict into the model
save_dir = 'Trained_models/'
model_filename = "VotingClassifier_Sequential_3_ckpt.pth"
load_path = os.path.join(save_dir, model_filename)

# Load model state dict
ensemble.load_state_dict(torch.load(load_path))


<All keys matched successfully>

In [9]:
# Evaluate the ensemble
acc = ensemble.evaluate(test_loader)  # testing accuracy
print(f"Test Accuracy: {acc:.2f}%")


Test Accuracy: 97.74%
